# Overview

In this notebook, we will use our own dataset as the knwoloedge of the RAG. We will load our data from CSV file. Next, loading `RagRetriever` witht the customise data. Finally, we will do some inference under the context of the customize data.


# Installing faiss(GPU+CPU)

Here we install Faceboook Faiss which is a library for efficient **similarity search** and **cluster of dense vectors**. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also contains supporting code for evaluation and parameter tuning. Some of the most useful algorithms are implemented on the GPU. Further more, the GPU implementation can accept input from either CPU or GPU memeory. 

In [3]:
%%capture
!conda install -c pytorch -c nvidia faiss-gpu=1.8.0 -y

Channels:
 - pytorch
 - nvidia
 - rapidsai
 - conda-forge
 - defaults
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.1.2

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - faiss-gpu=1.8.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cuda-cudart-11.8.89        |                0         197 KB  nvidia
    faiss-gpu-1.8.0            |py3.10_hedc54c9_0_cuda11.4.4         5.0 MB  pytorch
    libcublas-11.11.3.6        |                0       364.0 MB  nvidia
    libfaiss-1.8.0             |h5aaf3ed_0_cuda11.4.4       330.4 MB  pytorch
    mkl-2023.2.0               |   h84fe81f_50496       156.8 MB  conda-forge
    -----------------------------------------------------

In [10]:
import faiss

# Loading Dataset

The dataset needed for RAG must have two columns:
- title(string): title of the document
- text(string): text of a passage of the document

In [9]:
import pandas as pd

csv_path='/kaggle/input/knowledge-dataset/own_knwoledge_dataset.csv'

df=pd.read_csv(csv_path)
print(df.shape)

(3, 2)
